# Machine Learning Online Class - Exercise 4 Neural Network Learning

In [1]:
import numpy as np
from scipy.optimize import fmin_l_bfgs_b, fmin_cg
from scipy.io import loadmat
import random
import matplotlib.pyplot as plt

In [2]:
#Initialization
input_layer_size = 400
hidden_layer_size = 25
num_labels = 10

## =========== Part 1: Loading and Visualizing Data ============= 

In [3]:
data = loadmat("ex4data1.mat")
X = data['X']
y = data['y'] #Label for '0' is 10

In [4]:
#randomly select 100 data points to display
rand_indices = random.sample(range(X.shape[0]), 100)
sel = X[rand_indices,:]

In [5]:
#Display the data as images
fig0, ax = plt.subplots(10,10,figsize=(10, 10))
for i in range(10):
    for j in range(10):
        ax[i][j].imshow(sel[10*i+j].reshape(20,20, order='F'), cmap='gray')#order 'F' orders by column, like fortran

In [6]:
X = np.insert(X, 0, np.ones(X.shape[0]), axis=1) #add bias column
X.shape, y.shape

((5000, 401), (5000, 1))

## ================ Part 2: Loading Parameters ================

In [7]:
#We load some pre-initialized parameters
params = loadmat("ex4weights.mat")
Theta1 = params['Theta1']
Theta2 = params['Theta2']
Theta1.shape, Theta2.shape

((25, 401), (10, 26))

In [8]:
#Unroll parameters
nn_params = np.concatenate((Theta1.reshape(1,Theta1.shape[0]*Theta1.shape[1]),
                           Theta2.reshape(1,Theta2.shape[0]*Theta2.shape[1])), 
                          axis=1)[0]
nn_params.shape

(10285,)

## ================ Part 3: Compute Cost (Feedforward) ================

<img src="nnet.png" width=200/>
<br >
$$J(\Theta)=-\frac{1}{m}\sum_{\alpha=1}^{m}\sum_{i=1}^{n_L} \left[y_{i,\alpha}\log(a^{(L)}_{i})+(1-y_{i,\alpha})\log(1-a^{(L)}_{i}))\right]+\frac{\lambda}{2m}\sum_{k=1}^{L-1}\sum_{s=1}^{n_{k}}\sum_{r=1}^{n_{k+1}}\left(\Theta_{rs}^{(k)}\right)^2,$$
where
$$a_{i}^{(k)}=h(z^{(k)}_{i}),$$
$$a_{i}^{(1)}=x_{i,\alpha},\ 1\leq\alpha\leq m,$$ <br >
$$\Theta^{(k)}=\left(\begin{array}{cccc}
\Theta_{10}^{(k)} & \Theta_{11}^{(k)} & \cdots & \Theta_{1n_k}^{(k)} \\ 
\Theta_{20}^{(k)} & \Theta_{21}^{(k)} & \cdots & \Theta_{2n_k}^{(k)} \\
\vdots & \vdots & \ddots & \vdots \\
\Theta_{n_{k+1}0}^{(k)} & \Theta_{n_{k+1}1}^{(k)} & \cdots & \Theta_{n_{k+1}n_k}^{(k)} \\
\end{array}\right),$$
and
$$z^{(k)}_{i}=\sum_{j=0}^{n_{k-1}}\Theta^{(k-1)}_{ij}a^{(k-1)}_j,\ \ a^{(k-1)}_0=1.$$
If the sigmoid is used as activation function then
$$h(z)=\frac{1}{1+e^{-z}}.$$
NOTE: The vectors $y_{\alpha}$ are one-hot encodings for the different classes in the classification problem.

In [48]:
#Logistic function
def sigmoid(z):
    return 1.0 / (1 + np.exp(-z))

#Regularized neural network cost function (Forward propagation)
def nnJreg(nn_params, X, y, l, input_layer_size, hidden_layer_size, num_labels):
    #Retrieve the parameter matrices
    sz1 = hidden_layer_size*(input_layer_size+1)
    sz2 = num_labels*(hidden_layer_size+1)
    #print(sz1,sz2)
    T1 = nn_params[:sz1].reshape((hidden_layer_size,input_layer_size+1))
    T2 = nn_params[sz1:].reshape((num_labels,hidden_layer_size+1))
    #print(T1.shape, T2.shape)
    
    m = X.shape[0] #Number of data samples
    #Obtain layer 2 for all samples
    z2 = np.matmul(X,T1.T)
    a2 = sigmoid(z2)
    a2 = np.insert(a2,0,np.ones(a2.shape[0]), axis=1)
    #Obtain layer 3 (output) for all samples
    z3 = np.matmul(a2,T2.T)
    a3 = sigmoid(z3)
    #print(a3[0,:])
    
    #One-hot encoding for y in all samples
    ym = np.array([[1 if y[j]==i else 0 for i in [1,2,3,4,5,6,7,8,9,10]] for j in range(m)])
    #print(ym[0,:])
    
    #Copute J without regularization
    J = -(ym*np.log(a3) + (1-ym)*np.log(1-a3)).sum() / m
    
    #Compute the regularization term (delete the bias column)
    R = l * ((T1[:,1:]**2).sum() + (T2[:,1:]**2).sum()) / (2*m) 
    print(J+R)
    return J + R

In [10]:
l = 0
Jval = nnJreg(nn_params, X, y, l, input_layer_size, hidden_layer_size, num_labels)
print("Cost at parameters (loaded from ex4weights): {0:f}".format(Jval))
print("(this value should be about 0.287629)")

0.2876291651613189 [3.85613838e-04 9.68544083e-01 1.92134752e-03 1.38526834e-04
 3.20810992e-03 7.01713717e-04 6.45235792e-04 1.66993752e-02
 1.00700486e-01 3.25396474e-03]
Cost at parameters (loaded from ex4weights): 0.287629
(this value should be about 0.287629)


## =============== Part 4: Implement Regularization ===============

In [11]:
l = 1
Jval = nnJreg(nn_params, X, y, l, input_layer_size, hidden_layer_size, num_labels)
print("Cost at parameters (loaded from ex4weights): {0:f}".format(Jval))
print("(this value should be about 0.383770)")

0.38376985909092365 [3.85613838e-04 9.68544083e-01 1.92134752e-03 1.38526834e-04
 3.20810992e-03 7.01713717e-04 6.45235792e-04 1.66993752e-02
 1.00700486e-01 3.25396474e-03]
Cost at parameters (loaded from ex4weights): 0.383770
(this value should be about 0.383770)


## ================ Part 5: Sigmoid Gradient  ================

In [12]:
#Derivative of the logistic function
def grad_sigmoid(z):
    return sigmoid(z) * (1-sigmoid(z))

In [13]:
g = grad_sigmoid(np.array([-1,-0.5,0,0.5,1]))
print("Sigmoid gradient evaluated at [-1 -0.5 0 0.5 1]:")
print(g)

Sigmoid gradient evaluated at [-1 -0.5 0 0.5 1]:
[0.19661193 0.23500371 0.25       0.23500371 0.19661193]


## ================ Part 6: Initializing Pameters ================

In [14]:
#Parameters are initialized at random small values (between -ep and ep), otherwise all neurons 
# on a layer would be equal
ep = 0.12
ini_Theta1 = ep * (2*np.random.rand(hidden_layer_size,input_layer_size+1)-1)
ini_Theta2 = ep * (2*np.random.rand(num_labels,hidden_layer_size+1)-1)

#unroll parameters
ini_nn_params = np.concatenate((ini_Theta1.reshape(1,ini_Theta1.shape[0]*ini_Theta1.shape[1]),
                                ini_Theta2.reshape(1,ini_Theta2.shape[0]*ini_Theta2.shape[1])),
                               axis=1)[0]
ini_nn_params.shape

(10285,)

## =============== Part 7: Implement Backpropagation ===============

The elements of the gradient can be computed as
$$\frac{\partial J}{\partial\Theta_{ij}^{(k)}}=\left(\begin{array}{ccc}
\frac{\partial J}{\partial a_{1}^{(L)}} & \cdots &\frac{\partial J}{\partial a_{n_L}^{(L)}}\\
\end{array}\right)
\left(\begin{array}{ccc}
\frac{\partial a_{1}^{(L)}}{\partial a_{1}^{(L-1)}} & \cdots & \frac{\partial a_{1}^{(L)}}{\partial a_{n_{L-1}}^{(L-1)}}\\
\vdots & \ddots & \vdots \\
\frac{\partial a_{n_L}^{(L)}}{\partial a_{1}^{(L-1)}} & \cdots & \frac{\partial a_{n_L}^{(L)}}{\partial a_{n_{L-1}}^{(L-1)}}\\
\end{array}\right)\cdots
\left(\begin{array}{ccc}
\frac{\partial a_{1}^{(k+2)}}{\partial a_{1}^{(k+1)}} & \cdots & \frac{\partial a_{1}^{(k+2)}}{\partial a_{n_{k+1}}^{(k+1)}}\\
\vdots & \ddots & \vdots \\
\frac{\partial a_{n_{k+2}}^{(k+2)}}{\partial a_{1}^{(k+1)}} & \cdots & \frac{\partial a_{n_{k+2}}^{(k+2)}}{\partial a_{n_{k+1}}^{(k+1)}}\\
\end{array}\right)
\left(\begin{array}{c}
\frac{\partial a_{1}^{(k+1)}}{\partial\Theta_{ij}^{(k)}}\\
\vdots\\
\frac{\partial a_{n_{k+1}}^{(k+1)}}{\partial\Theta_{ij}^{(k)}}\\
\end{array}\right)+\frac{\lambda}{m}\Theta_{ij}^{(k)}(1-\delta_{j0}),$$
where
$$\frac{\partial J}{\partial a_i^{(L)}}=-\frac{1}{m}\sum_{\alpha=1}^{m}\left(\frac{y_{i,\alpha}}{a_i^{(L)}}-\frac{1-y_{i,\alpha}}{1-a_i^{(L)}}\right)=\frac{1}{m}\sum_{\alpha=1}^{m}\frac{(a_i^{(L)}-y_{i,\alpha})}{a_i^{(L)}(1-a_i^{(L)})},$$
$$\frac{\partial a_{r}^{(k+1)}}{\partial a_{s}^{(k)}}=\frac{\partial h}{\partial z_r^{(k+1)}}\frac{\partial z_r^{(k+1)}}{\partial a_{s}^{(k)}}=h'(z_r^{(k+1)})\Theta_{rs}^{(k)},$$
and
$$\frac{\partial a_{r}^{(k+1)}}{\partial\Theta_{ij}^{(k)}}=\frac{\partial h}{\partial z_r^{(k+1)}}\frac{\partial z_r^{(k+1)}}{\partial\Theta_{ij}^{(k)}}=h'(z_r^{(k+1)})a_j^{(k)}\delta_{ri}.$$
NOTE: $a_j^{(k)}:=a_{j,\alpha}^{(k)}$, where $\alpha$ denotes the sample.

We have then that for $k=L-1$
$$\frac{\partial J}{\partial\Theta_{ij}^{(L-1)}}=\sum_{l=1}^{n_{L}}\frac{\partial J}{\partial a_l^{(L)}}\frac{\partial a_l^{(L)}}{\partial \Theta_{ij}^{(L-1)}}=\frac{1}{m}\sum_{\alpha=1}^{m}\frac{(a_i^{(L)}-y_{i,\alpha})}{a_i^{(L)}(1-a_i^{(L)})}h'(z_i^{(L)})a_j^{(L-1)}=\frac{1}{m}\sum_{\alpha=1}^{m}\delta_i^{(L)}h'(z_i^{(L)})a_j^{(L-1)},$$
while for, say, $k=L-3$
\begin{align}
\frac{\partial J}{\partial\Theta_{ij}^{(L-3)}}&=\sum_{s=1}^{n_{L-2}}\sum_{r=1}^{n_{L-1}}\sum_{l=1}^{n_{L}}\frac{\partial J}{\partial a_l^{(L)}}\frac{\partial a_l^{(L)}}{\partial a_r^{(L-1)}}\frac{\partial a_r^{(L-1)}}{\partial a_s^{(L-2)}}\frac{\partial a_s^{(L-2)}}{\partial \Theta_{ij}^{(L-3)}}\\
&=\frac{1}{m}\sum_{\alpha=1}^{m}\underbrace{\sum_{r=1}^{n_{L-1}}\underbrace{\sum_{l=1}^{n_{L}}\underbrace{\frac{(a_l^{(L)}-y_{l,\alpha})}{a_l^{(L)}(1-a_l^{(L)})}}h'(z_l^{(L)})\Theta_{lr}^{(L-1)}}h'(z_r^{(L-1)})\Theta_{ri}^{(L-2)}}h'(z_i^{(L-2)})a_j^{(L-3)}\\
&=\frac{1}{m}\sum_{\alpha=1}^{m}\delta_i^{(L-2)}h'(z_i^{(L-2)})a_j^{(L-3)},
\end{align}
that is, in general
$$\frac{\partial J}{\partial\Theta_{ij}^{(k)}}=\frac{1}{m}\sum_{\alpha=1}^{m}\delta_i^{(k+1)}h'(z_i^{(k+1)})a_j^{(k)}+\frac{\lambda}{m}\Theta_{ij}^{(k)}(1-\delta_{j0}),$$
where
$$\delta_r^{(k)}=\sum_{l=1}^{n_{k+1}}\delta_l^{(k+1)}h'(z_l^{(k+1)})\Theta_{lr}^{(k)},$$
and
$$\delta_s^{(L)}=\frac{(a_s^{(L)}-y_{s,\alpha})}{a_s^{(L)}(1-a_s^{(L)})}.$$
In the case where a sigmoid is used as activation function
$$h'(z_r^{(k)})=h(z_r^{(k)})(1-h(z_r^{(k)}))=a_r^{(k)}(1-a_r^{(k)}).$$


In [15]:
#Gradient of the cost function (Not efficient because the a's are computed again)
def grad_nnJreg(nn_params, X, y, l, input_layer_size, hidden_layer_size, num_labels):
    #Retrieve the parameter matrices
    sz1 = hidden_layer_size*(input_layer_size+1)
    sz2 = num_labels*(hidden_layer_size+1)
    T1 = nn_params[:sz1].reshape((hidden_layer_size,input_layer_size+1))
    T2 = nn_params[sz1:].reshape((num_labels,hidden_layer_size+1))
    
    #Feedforward to compute a's---------------------------------------------
    m = X.shape[0] #Number of data samples
    #Obtain layer 2 for all samples
    z2 = np.matmul(X,T1.T)
    a2 = sigmoid(z2)
    a2 = np.insert(a2,0,np.ones(a2.shape[0]), axis=1)
    #Obtain layer 3 (output) for all samples
    z3 = np.matmul(a2,T2.T)
    a3 = sigmoid(z3)
    #print(a2.shape)
    
    #One-hot encoding for y in all samples
    ym = np.array([[1 if y[j]==i else 0 for i in [1,2,3,4,5,6,7,8,9,10]] for j in range(m)])
    
    #Backpropagation to compute gradient------------------------------------
    #Compute delta vector for the last layer (L) for all samples (each row is a sample)
    d3 = (a3 - ym) / (a3*(1-a3))
    d3h = d3 * grad_sigmoid(z3) 
    #print(d3h.shape)
    #Compute gradient with respect to Theta(L-1)
    T2grad = np.zeros((num_labels,hidden_layer_size+1))
    for i in range(m):
        T2grad += np.tensordot(d3h[i],a2[i],axes=0) 
    T2grad[:,1:] += l*T2[:,1:] #add regularization term
    T2grad /= m
    
    #Compute delta vector for layer (L-1) for all samples (each row is a sample)
    d2 = np.zeros((m,hidden_layer_size+1))
    for i in range(m):
        d2[i] = np.matmul(d3h[i],T2)
    d2h = d2[:,1:] * grad_sigmoid(z2)
    #print(d2h.shape)
    #Compute gradient with respect to Theta(L-2)
    T1grad = np.zeros((hidden_layer_size,input_layer_size+1))
    for i in range(m):
        T1grad += np.tensordot(d2h[i],X[i],axes=0)
    T1grad[:,1:] += l*T1[:,1:] #add regularization term
    T1grad /= m
    
    return np.concatenate((T1grad.reshape(1,T1grad.shape[0]*T1grad.shape[1]),
                           T2grad.reshape(1,T2grad.shape[0]*T2grad.shape[1])), 
                          axis=1)[0]


#Compute the gradient using finite differences
def test_grad(nn_params, X, y, l, input_layer_size, hidden_layer_size, num_labels, mat, i, j):
    sz1 = hidden_layer_size*(input_layer_size+1)
    m = X.shape[0] #Number of data samples
    epsilon = 1.e-4
    aux = nn_params[:]
    
    val = 0
    if mat == 1:
        aux[i*(input_layer_size+1)+j] -= epsilon
        Ji = nnJreg(aux, X, y, l, input_layer_size, hidden_layer_size, num_labels)
        aux[i*(input_layer_size+1)+j] += 2*epsilon
        Jf = nnJreg(aux, X, y, l, input_layer_size, hidden_layer_size, num_labels)
        aux[i*(input_layer_size+1)+j] -= epsilon
        val = (Jf-Ji) / (2*epsilon)
        #print("T1({0},{1}): {2}".format(i,j,val))
    
    if mat == 2:
        aux[sz1+i*(hidden_layer_size+1)+j] -= epsilon
        Ji = nnJreg(aux, X, y, l, input_layer_size, hidden_layer_size, num_labels)
        aux[sz1+i*(hidden_layer_size+1)+j] += 2*epsilon
        Jf = nnJreg(aux, X, y, l, input_layer_size, hidden_layer_size, num_labels)
        aux[sz1+i*(hidden_layer_size+1)+j] -= epsilon
        val = (Jf-Ji) / (2*epsilon)
        #print("T2({0},{1}): {2}".format(i,j,val))
        
    return val

In [16]:
l = 0
Jgrad = grad_nnJreg(nn_params, X, y, l, input_layer_size, hidden_layer_size, num_labels)
Jgrad.shape

(10285,)

In [17]:
#Test if the gradient is correct
a = np.random.randint(hidden_layer_size)
b = np.random.randint(input_layer_size+1)
mat = 1
Jgrad_val = test_grad(nn_params, X, y, l, input_layer_size, hidden_layer_size, num_labels,mat,a,b)
print(Jgrad[a*(input_layer_size+1)+b], Jgrad_val)
print("err({0},{1}): {2}".format(a,b,(Jgrad[a*(input_layer_size+1)+b]-Jgrad_val)/Jgrad_val))

a = np.random.randint(num_labels)
b = np.random.randint(hidden_layer_size+1)
mat = 2
Jgrad_val = test_grad(nn_params, X, y, l, input_layer_size, hidden_layer_size, num_labels,mat,a,b)
sz1 = hidden_layer_size*(input_layer_size+1)
print(Jgrad[sz1+a*(hidden_layer_size+1)+b], Jgrad_val)
print("err({0},{1}): {2}".format(a,b,(Jgrad[sz1+a*(hidden_layer_size+1)+b]-Jgrad_val)/Jgrad_val))

0.2876291616080891 [3.85608810e-04 9.68543982e-01 1.92130171e-03 1.38525556e-04
 3.20816093e-03 7.01721383e-04 6.45230013e-04 1.66996881e-02
 1.00702210e-01 3.25401696e-03]
0.2876291687555456 [3.85618866e-04 9.68544184e-01 1.92139334e-03 1.38528113e-04
 3.20805892e-03 7.01706051e-04 6.45241572e-04 1.66990624e-02
 1.00698762e-01 3.25391253e-03]
3.573728199519871e-05 3.573728257766362e-05
err(13,156): -1.6298522660271803e-08
0.2876292213560657 [3.85613838e-04 9.68544083e-01 1.92134752e-03 1.38526834e-04
 3.20810992e-03 7.01713717e-04 6.45235792e-04 1.66993752e-02
 1.00700368e-01 3.25396474e-03]
0.28762910902045885 [3.85613838e-04 9.68544083e-01 1.92134752e-03 1.38526834e-04
 3.20810992e-03 7.01713717e-04 6.45235792e-04 1.66993752e-02
 1.00700604e-01 3.25396474e-03]
-0.000561678035396485 -0.0005616780343609129
err(8,2): 1.8437112791150048e-09


## =============== Part 8: Implement Regularization ===============

In [18]:
l = 1
Jgrad = grad_nnJreg(nn_params, X, y, l, input_layer_size, hidden_layer_size, num_labels)
Jgrad.shape

(10285,)

In [19]:
#Test if the gradient is correct
a = np.random.randint(hidden_layer_size)
b = np.random.randint(input_layer_size+1)
mat = 1
Jgrad_val = test_grad(nn_params, X, y, l, input_layer_size, hidden_layer_size, num_labels,mat,a,b)
print(Jgrad[a*(input_layer_size+1)+b], Jgrad_val)
print("err({0},{1}): {2}".format(a,b,(Jgrad[a*(input_layer_size+1)+b]-Jgrad_val)/Jgrad_val))

a = np.random.randint(num_labels)
b = np.random.randint(hidden_layer_size+1)
mat = 2
Jgrad_val = test_grad(nn_params, X, y, l, input_layer_size, hidden_layer_size, num_labels,mat,a,b)
sz1 = hidden_layer_size*(input_layer_size+1)
print(Jgrad[sz1+a*(hidden_layer_size+1)+b], Jgrad_val)
print("err({0},{1}): {2}".format(a,b,(Jgrad[sz1+a*(hidden_layer_size+1)+b]-Jgrad_val)/Jgrad_val))

0.38376986046290307 [3.85613838e-04 9.68544083e-01 1.92134752e-03 1.38526834e-04
 3.20810992e-03 7.01713717e-04 6.45235792e-04 1.66993752e-02
 1.00700486e-01 3.25396474e-03]
0.38376985772148153 [3.85613838e-04 9.68544083e-01 1.92134752e-03 1.38526834e-04
 3.20810992e-03 7.01713717e-04 6.45235792e-04 1.66993752e-02
 1.00700486e-01 3.25396474e-03]
-1.370710778266806e-05 -1.3707107671123708e-05
err(14,279): 8.137701517129814e-09
0.38376989755719626 [3.85613838e-04 9.68544083e-01 1.92134752e-03 1.38526834e-04
 3.20810992e-03 7.01705832e-04 6.45235792e-04 1.66993752e-02
 1.00700486e-01 3.25396474e-03]
0.3837698206801474 [3.85613838e-04 9.68544083e-01 1.92134752e-03 1.38526834e-04
 3.20810992e-03 7.01721602e-04 6.45235792e-04 1.66993752e-02
 1.00700486e-01 3.25396474e-03]
-0.00038438524529766315 -0.0003843852441898221
err(5,16): 2.8821112391742006e-09


## =================== Part 9: Training NN ===================

In [49]:
l = 1
fin_nn_params = fmin_cg(nnJreg, ini_nn_params, grad_nnJreg, args=(X,y,l,input_layer_size, hidden_layer_size, num_labels),maxiter=100)
fin_nn_params.shape

6.86187653305713
4.2947528374584705
3.291226518983927
3.2485551644949533
3.2407312949082616
3.2112122871017337
3.124192516455245
3.093297848196346
2.8166075927809184
2.5013635658980746
2.2979570056686907
2.2012962521682793
2.0044428216411867
1.952326332158628
1.7701674755810723
1.6458634993980266
1.5295943030150356
1.478410743047159
1.4306175589058767
1.3358943475094636
1.306777706478238
1.2832451120591089
1.2434699488974603
1.2049556959673897
1.1607518376881538
1.119083457577239
1.1043117626774204
1.0874009970175014
1.0584398611806494
1.0273933345597375
1.0100474643322592
0.970810128732432
0.9620579780491991
0.9424518430435866
0.932634726540427
0.9210920113063782
0.9105932800709822
0.8842490034793871
0.8709903266443884
0.8617856060633409
0.8518191177658041
0.8356451454595941
0.8300497158783222
0.8182698298825849
0.8099956916398373
0.7925978729642698
0.7846479044089099
0.7702718910527225
0.7639171977506567
0.7455399171212099
0.7384897762768573
0.7257182372107964
0.7034864315777541
0.69

(10285,)

In [50]:
fin_nn_params
#print("J: {0}".format(fin_nn_params[1]))
#print(fin_nn_params[0])
#print(fin_nn_params[2])

array([-0.53436812,  0.05471417, -0.03745991, ...,  0.5892818 ,
       -0.38025807,  0.55044569])

In [24]:
#Out = open("fin_nn_params.dat", "w")
#for q in fin_nn_params:
#    Out.write("{0} ".format(q))
#Out.close()

In [ ]:
#f_nn_params = np.array([0 for i in range(ini_nn_params.shape[0])])
#In = open("fin_nn_params.dat", "r")
#dat = In.readline().split()
#for i in range(f_nn_params.shape[0]):
#    f_nn_params[i] = float(dat[i])
#In.close()
#f_nn_params.shape

In [51]:
sz1 = hidden_layer_size*(input_layer_size+1)
Theta1 = fin_nn_params[:sz1].reshape((hidden_layer_size,input_layer_size+1))
Theta2 = fin_nn_params[sz1:].reshape((num_labels,hidden_layer_size+1))
Theta1.shape, Theta2.shape

((25, 401), (10, 26))

## ================= Part 10: Visualize Weights =================

In [52]:
#Display the data as images
fig1, ax = plt.subplots(5,5,figsize=(5, 5))
for i in range(5):
    for j in range(5):
        ax[i][j].imshow(Theta1[5*i+j,1:].reshape(20,20), cmap='gray')

## ================= Part 11: Implement Predict =================

In [53]:
#Function to predict all the samples in the training set with the trained model
def predict(T1, T2, X, id):
    z2 = np.matmul(X,T1.T)
    a2 = sigmoid(z2)
    a2 = np.insert(a2,0,np.ones(a2.shape[0]), axis=1)
    #Obtain layer 3 (output) for all samples
    z3 = np.matmul(a2,T2.T)
    a3 = sigmoid(z3)
    #print(a3[id,:])
    
    
    pp = np.array([[np.where(row == np.max(row))[0][0]] for row in a3])
    p = np.array([i+1 for i in pp])
    return p

In [54]:
id = 100
pred = predict(Theta1, Theta2, X, id)
pred.shape

(5000, 1)

[10]


In [55]:
print(y[id])
fig1, ax = plt.subplots(1,1,figsize=(1, 1))
ax.imshow(X[id,1:].reshape(20,20, order='F'), cmap='gray')#order 'F' orders by column, like fortran

In [56]:
print("Training set accuracy: {0:.3f}".format(np.mean(pred==y)))

Training set accuracy: 0.979


_____________________________________________________________________________
_____________________________________________

In [ ]:
#Reshape and concatenate several matrices into a single long vector:
import numpy as np

T1 = np.ones((10,11))
T2 = 2*np.ones((10,11))
T3 = 3*np.ones((1,11))

Tvec = np.concatenate((T1.reshape(1,T1.shape[0]*T1.shape[1]),
                       T2.reshape(1,T2.shape[0]*T2.shape[1]),
                       T3.reshape(1,T3.shape[0]*T3.shape[1])),
                      axis=1)
Tvec

In [ ]:
#Get matrices back
ind1 = T1.shape[0]*T1.shape[1]
ind2 = T1.shape[0]*T1.shape[1]+T2.shape[0]*T2.shape[1]
Tvec[0][ind1:ind2].reshape((T2.shape[0],T2.shape[1]))

In [ ]:
#Create a random matrix
e = 0.2
R = e * (2*np.random.rand(3,4)-1)
R